In [2]:
import polars as pl
from functools import partial
from itertools import product
import multiprocessing
from numba import jit

In [3]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/SP500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])
    #stockComponents.append(constituents[k,0]+'.L')

In [4]:
#Load prices and timestamps from CSVs
currentDf=pl.read_csv("../Data/SP500/dailyHist2000/TSLA.csv", infer_schema_length=None)
openPrices=currentDf['Open'].to_numpy()
closePrices=currentDf['Close'].to_numpy()
timestamps=currentDf['timestamp'].to_numpy()

In [5]:
@jit(nopython=True)
def algo(paramsList, openPrices, closePrices, timestamps):

    change=paramsList[0]
    takeProfit=paramsList[1]
    stopLoss=paramsList[2]
    timeout=paramsList[3]
    daysBefore=paramsList[4]
    fee=paramsList[5]
    
    tradePercent=0

    total=0
    totalTime=0

    start=daysBefore
    end=int(len(openPrices)-((timeout/86400)+1))

    for j in range(start, end):
        beforeTime=0
        newTime=j-daysBefore
        while beforeTime<(timestamps[j]-daysBefore*86400):
            beforeTime=timestamps[newTime]
            newTime+=1

        if openPrices[j]<openPrices[j-daysBefore]*(2-change):
            total+=1
            buyPrice = openPrices[j]

            startTime=timestamps[j]

            time=0
            ratio=1

            datapoint=j+1
            while time < timeout and stopLoss<ratio<takeProfit:
                time = int(timestamps[datapoint])-startTime
                sellPrice=closePrices[datapoint]
                ratio = sellPrice/buyPrice
                datapoint+=1
            # while datapoint<(j+timeout) and stopLoss<ratio<takeProfit:
            #     sellPrice=closePrices[stock][datapoint]
            #     ratio = sellPrice/buyPrice
            #     datapoint+=1
            
            tradePercent+=ratio-fee
            totalTime+=time
        
    try:
        tradePercent=tradePercent/total
    except:
        pass

    averageTime=((totalTime/86400)/total)+1
    tradePerDay = (tradePercent**(1/averageTime))
    returnList=[tradePerDay, total, paramsList[0], paramsList[1], paramsList[2], paramsList[3], paramsList[4], paramsList[5]]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)
partialAlgo = partial(algo, openPrices=openPrices, closePrices=closePrices, timestamps=timestamps)

In [6]:
#change
minimum=1.00
maximum=1.2
difference=0.02
changeList=[]
j=minimum
while j<maximum+difference:
    changeList.append(j)
    j+=difference

#takeProfit
minimum=1.05
maximum=1.50
difference=0.05
takeProfitList=[]
j=minimum
while j<maximum+difference:
    takeProfitList.append(j)
    j+=difference

#stopLoss
minimum=0.50
maximum=0.95
difference=0.05
stopLossList=[]
j=minimum
while j<maximum+difference:
    stopLossList.append(j)
    j+=difference

#timeout
timeoutList=[]
dayList=[1,2,3,4,5,10,20,30,60]
for day in dayList:
    timeoutList.append(day*86400)

#daysBefore
daysBeforeList=[1,2,3,4,5,10,20,30,60]

#minimum total for algo
minTotal=200

#rough commission fee
fee=[0.01]

fullCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, daysBeforeList, fee))
len(fullCombinations)

89100

In [7]:
algo(fullCombinations[0], openPrices, closePrices, timestamps)
#fullCombinations[0]

[0.997029438123144, 1565.0, 1.0, 1.05, 0.5, 86400.0, 1.0, 0.01]

In [69]:
%%timeit -n5
algo(fullCombinations[0], openPrices, closePrices, timestamps)

10 µs ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [70]:
%%timeit -n5
partialAlgo(fullCombinations[0])

10.2 µs ± 3.43 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [71]:
results=[]
orderCombinations=[]
with multiprocessing.Pool(20) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        if result[1]>=minTotal:
            results.append(result[0])
            orderCombinations.append((result[2],result[3],result[4],int(result[5]),int(result[6]),result[7]))


In [72]:
sortedResults, resultCombinations = zip(*sorted(zip(results, orderCombinations), reverse=True))
partialAlgo(resultCombinations[0])
resultCombinations[0]

(1.2000000000000002, 1.5000000000000004, 0.55, 2592000, 30, 0.01)

In [73]:
bestStrats=[]
for topResult in range(200):
    bestStrats.append([resultCombinations[topResult],partialAlgo(resultCombinations[topResult])])

In [74]:
bestStrats[0]

[(1.2000000000000002, 1.5000000000000004, 0.55, 2592000, 30, 0.01),
 [1.0034407938638268,
  225.0,
  1.2000000000000002,
  1.5000000000000004,
  0.55,
  2592000.0,
  30.0,
  0.01]]